### DB 에서 가져온 데이터( 레시피 )로 유사도 분석

In [13]:
import pandas as pd
import mariadb
import numpy as np
from tqdm import tqdm

In [4]:
# mariaDB 접속값
conn_params = {'host':'172.20.58.217','port':3306,'user':'project_user',
               'password':'project_user','database':"project",'autocommit':True}
conn = mariadb.connect(**conn_params)
cursor = conn.cursor()

# DB 에서 tb_recipe  데이터 조회하는  sql 실행
cursor.execute('select * from tb_recipe;')

# 조회결과 가져오기
queryDatas=cursor.fetchall()

df=pd.DataFrame(queryDatas)

cursor.close()
conn.close()

In [7]:
# 가져온 df 확인
df.head()

,0,1
0,BLT샌드위치,기름없는 팬에 베이컨을 올리고 약불에서 바삭하게 구워주세요|팬에 베이컨 기름을 버리...
1,KFC비스켓,"박력분 1컵에 베이킹파우더 1/3스푼, 설탕 2스푼, 소금 1/3스푼을 섞어 체망에..."
2,KFC코울슬로,계랑은 밥숟가락 기준입니다!!|- 피망과 양파는 깨꼿히 씻어서|- 잘게 다져준다.|...
3,LA갈비구이,갈비는 찬물에 담가 핏물을 빼고 손으로 주물러 흔들어 씻어뼛가루를 제거한다.물기를 ...
4,가나슈초코케이크,달걀을 풀어준 다음 설탕을 3회정도 나눠 넣으며 거품을 올려주세요. 밑에 따뜻한 물...


In [8]:
# df 컬럼 이름 설정
df.columns=['cook_name','recipe']

In [16]:
from konlpy.tag import Okt

okt=Okt()

# 토큰화
total=[]
for i in tqdm(df['recipe']):
    t=okt.nouns(i)
    total.append(" ".join(t))

100%|██████████| 4963/4963 [00:41<00:00, 119.81it/s]


In [23]:
# 명사만 있는 새로운 컬럼 추가
df['n_recipe']=total

df.tail()

,cook_name,recipe,n_recipe
4958,흑임자레이어무스케이크,"오레오를 잘게 부순 뒤, 녹인 버터를 넣고 잘 섞어 주세요|무스틀 바닥에 넣고 단단...",오레오 부순 뒤 버터 무스 틀 바닥 흑임자 무스 동안 냉장고 크림 치즈 풀 설탕 플...
4959,흑임자머핀,오일 + 식초 + 두유 + 올리고당 + 설탕을 넣고 잘 섞어 주세요|통밀 + 흑임자...,오일 식초 두유 올리고당 설탕 통밀 흑임자 베이 킹소 아몬드 파우더 체 반죽 유산 ...
4960,흑임자어묵조림,"흑임자어묵을 끓는물에 잠깐 데쳐주었어요.|맛간장 2, 꿀 2, 물 50ml, 마늘 ...",흑임자 어묵 물 잠깐 데 맛 간장 꿀 물 마늘 청주 생강 즙 참기름 조림 조림 흑임...
4961,흰살생선뭇국,"물 700ml에 육수를 내주세요저는 멸치향이 강해 디포리로 대신하고다시마, 대파뿌리...",물 육수 저 멸치 향 디포 리 대신 다시마 대파 뿌리 양파 껍질 육수 육수 총 정도...
4962,히야시추카,먼저 청오이는 반으로 갈라주고 수저로 가운데 씨 부분으 빼줍니다.|오이를 채쳐주세요...,먼저 청 오이 반 수저 가운데 씨 부분 빼 오이 채 계란 체 알끈 팬 기름 체 계란...


In [64]:
# 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()
test=df["n_recipe"].to_list()
dtm=cv.fit_transform(test)
dtm=pd.DataFrame(dtm.toarray(),columns=cv.get_feature_names_out())
print(cv.get_feature_names_out())
dtm

['가감' '가게' '가격' ... '힘껏' '힘좀' '힘줄']


,가감,가게,가격,가기,가끔,가나,가나슈,가능,가니쉬,가닥,...,희망이,흰밥,흰색,흰자,흰자위,히트,히히히,힘껏,힘좀,힘줄
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b)) 


In [68]:
# 테스트 값
text = '감자와 치즈를 넣어서'
text = okt.nouns(text)
text = " ".join(text)

dtm_2=cv.transform([text])
target=dtm_2.toarray()[0]

In [70]:
box=[]
for i in range(len(dtm)):
    sample=dtm.iloc[i].to_numpy()
    cosine=make_cosine(target,sample)
    box.append(cosine)

# 데이터프레임에 유사도 열 추가
df['유사도분석']=box

# 유사도에 따라 데이터프레임 정렬
df.sort_values('유사도분석',ascending=False,inplace=True)
df.head(10)

,cook_name,recipe,n_recipe,유사도분석
1542,떡감자,감자가 빨리 익을 수 있도록 삶기 전에 미리 썰어줍니다.|냄비에 물을 붓고 감자를 ...,감자 익 수 전 미리 냄비 물 감자 감자 전분 소금 설탕 깨 감자 정도 손 반죽 손...,0.741249
1536,떠먹는치즈감자,우선 감자 껍질을 벗겨주세요!|감자가 빨리 익을 수 있도록 썰어주세요!|그리고 감자...,우선 감자 껍질 감자 익 수 감자 감자 포크 깨 소금 후추 마요네즈 우유 모짜렐라 ...,0.737865
3682,오지치즈감자,감자는 스틱 또는 깍둑썰기해서 찬 물에 약 10분간 놔둬 전분기를 빼 준 뒤 저처럼...,감자 스틱 둑 찬 물 약 분기 빼 준 뒤 저 기름 구 듯 분 미리 번 살짝 데 뒤 ...,0.717958
227,감자치즈호떡,감자를 껍질을 깍아 준비합니다|4등분으로 자르고|스틱 치즈는 12조각으로 자르고|찜...,감자 껍질 깍 준비 스틱 치즈 감자 약 내외 찌 감자 볼 더 체 설탕 스푼 소금 반...,0.714435
3228,아코디언감자구이,"재료:감자1개,베이컨3줄,체다치즈3개, 버터,허브솔트|체다치즈와 베이컨을 반으로 잘...",재료 감자 개 베이컨 줄 체다 치즈 개 버터 허브 솔트 체다 치즈 베이컨 반 준비 ...,0.673520
226,감자치즈피자,"감자,양파는 껍질을 벗겨주고 채썰어 준비해 둡니다.감자는 가늘게 채썰어주세요.|달걀...",감자 양파 껍질 채 준비 감자 채 달걀 개 대파 소금 고루 간 정도 팬 식용유 양파...,0.671156
1874,명란치즈감자,"감자를 완전히 익을때까지 삶아주세요|큰 그릇에 옮겨담아 잘 으깨줍니다|마요네즈, 후...",감자 익 때 그릇 깨 마요네즈 후추 준뒤 명란 한번 더 그릇 명란 감자 뒤 치즈 치...,0.668153
231,감자캐서롤,감자를 씻고 껍질을 벗겨서 준비해줍니다.|감자를 깍뚝썰기해줍니다.|끓는 물에서 소금...,감자 껍질 준비 감자 뚝 물 소금 감자 베이컨 준비 우유 마요네즈 사워크림 무염 버...,0.667424
239,감자파이,"끓는 물에 감자를 넣고, 완전히 익을 때까지 약 30분간 삶아주세요.|볼에 따뜻한 ...",물 감자 익 때 약 볼 상태 감자 포크 달걀 물 틀 감자 그 위 달걀 물 치즈 로 ...,0.648886
4194,치즈감자그라탕,"감자는 살짝 도톰하게 썰어주고,양파는 채썰기, 베이컨은 먹기 좋게 썰어줍니다.|감자...",감자 살짝 양파 채 베이컨 먹기 감자 물 차례 데 사용 팬 오일 버터 를 양파 베이...,0.648204


In [72]:
# 유사한 상위 3개 확인
top_3_similar = df.head(3)
top_3_similar[['cook_name','n_recipe','유사도분석']]

,cook_name,n_recipe,유사도분석
1542,떡감자,감자 익 수 전 미리 냄비 물 감자 감자 전분 소금 설탕 깨 감자 정도 손 반죽 손...,0.741249
1536,떠먹는치즈감자,우선 감자 껍질 감자 익 수 감자 감자 포크 깨 소금 후추 마요네즈 우유 모짜렐라 ...,0.737865
3682,오지치즈감자,감자 스틱 둑 찬 물 약 분기 빼 준 뒤 저 기름 구 듯 분 미리 번 살짝 데 뒤 ...,0.717958
